## Ingestion del archivo JSON multi-linhas "language_role.json"

#### Paso 0 - Actualización de variables y funciones

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/bronze

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
#Definindo o schema na variavel name_schema
languages_roles = StructType(fields=[
    StructField("roleId", IntegerType(), True),
    StructField("languageRole", StringType(), True)
])

In [0]:
languages_roles_df = spark.read \
               .schema(languages_roles) \
               .option("multiLine", True) \
               .json(f"{bronze_folder_path}/{v_file_date}/language_role.json")

In [0]:
#display(languages_roles_df)

#### Paso 2 - Renombrar el nombre de las columnas y añadir "ingestion_date" y "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
languages_roles_final_df = add_ingestion_date(languages_roles_df) \
                    .withColumnsRenamed({"roleId": "role_id", 
                                         "languageRole": "language_role"}) \
                    .withColumn("enviroment", lit(v_environment)) \
                    .withColumn("file_date", lit(v_file_date))

In [0]:
#display(languages_roles_final_df)

#### Paso 4 - Escribir datos en el datalake en formato parquet

In [0]:
#languages_roles_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/languages_roles/")
#languages_roles_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages_roles")
languages_roles_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages_roles")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/languages_roles

In [0]:
#display(spark.read.format('delta').load(f"{silver_folder_path}/languages_roles"))
#spark.sql("select * from movie_silver.languages_roles").display()

In [0]:
dbutils.notebook.exit("Success")